Install libraries

In [1]:
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.8 MB/s eta 0:00:00


In [2]:
from collections import defaultdict
import gdown
import gzip
import json
import random
import pickle

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import ticker
sns.set(style='ticks', font_scale=1.2)

In [3]:
#import the libraries for importing and dealing with data
import numpy as np
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#import libraries for processing
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [6]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Load German Dataset and Clean it**

In [7]:
#load and read the training TEXT From file
germantrainingSet = pd.read_csv('drive/MyDrive/germanDeeplTrans.txt', on_bad_lines='skip', sep=';', header=None)

In [8]:
#rename the column 2 to 'Tweet' and column 4 to 'Label
germantrainingSet=germantrainingSet.rename(columns={0: 'Tweet'})

In [9]:
germantrainingSet.head()

,Tweet
0,"@corinnamilborn Dear Corinna, we'd love to hav..."
1,@Martin28a You are indeed correct. Our tweet w...
2,@ahrens_theo happy greetings from the most bea...
3,@dushanwegner Yanks would have voted for anyth...
4,@spdde not a reliable negotiating partner. Rec...


In [10]:
def check_for_abuse(row):
    if row.endswith("ABUSE"):
        return 1
    else:
        return 0

In [11]:
germantrainingSet['Label'] = germantrainingSet['Tweet'].apply(check_for_abuse)

In [12]:
germantrainingSet['Tweet'] = germantrainingSet['Tweet'].apply(lambda x: ' '.join(x.split()[:-1]))

In [13]:
germantrainingSet['Tweet'] = germantrainingSet['Tweet'].str.replace('OTHER', '')
germantrainingSet['Tweet'] = germantrainingSet['Tweet'].str.replace('OFFENSE', '')

In [14]:
germantrainingSet.head()

,Tweet,Label
0,"@corinnamilborn Dear Corinna, we'd love to hav...",0
1,@Martin28a You are indeed correct. Our tweet w...,0
2,@ahrens_theo happy greetings from the most bea...,0
3,@dushanwegner Yanks would have voted for anyth...,0
4,@spdde not a reliable negotiating partner. Rec...,0


In [15]:
germantrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4816 entries, 0 to 4815
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   4816 non-null   object
 1   Label   4816 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 75.4+ KB


In [16]:
count = (germantrainingSet['Label'] == 1).sum()

# The condition (df['Label'] == 1) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 1 in 'Label' column:", count)

Number of rows with value 1 in 'Label' column: 994


In [17]:
countZero = (germantrainingSet['Label'] == 0).sum()

# The condition (df['Label'] == 0) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 0 in 'Label' column:", countZero)

Number of rows with value 0 in 'Label' column: 3822


In [18]:
#function remove_urls cleans the Urls from the Tweets and put in a seperate column
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

germantrainingSet["tweet_without_url"] = germantrainingSet["Tweet"].apply(lambda text: remove_urls(text))

In [19]:
germantrainingSet.head()

,Tweet,Label,tweet_without_url
0,"@corinnamilborn Dear Corinna, we'd love to hav...",0,"@corinnamilborn Dear Corinna, we'd love to hav..."
1,@Martin28a You are indeed correct. Our tweet w...,0,@Martin28a You are indeed correct. Our tweet w...
2,@ahrens_theo happy greetings from the most bea...,0,@ahrens_theo happy greetings from the most bea...
3,@dushanwegner Yanks would have voted for anyth...,0,@dushanwegner Yanks would have voted for anyth...
4,@spdde not a reliable negotiating partner. Rec...,0,@spdde not a reliable negotiating partner. Rec...


**Load Spanish Dataset and Clean it**

In [20]:
#load and read the training TEXT From file
spanishtrainingSet = pd.read_csv('drive/MyDrive/spanishDeeplTrans.txt', on_bad_lines='skip', sep=';', header=None)

In [21]:
spanishtrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5835 entries, 0 to 5834
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5835 non-null   object 
 1   1       5835 non-null   object 
 2   2       5835 non-null   object 
 3   3       5835 non-null   object 
 4   4       5835 non-null   float64
dtypes: float64(1), object(4)
memory usage: 228.1+ KB


In [22]:
spanishtrainingSet.head

<bound method NDFrame.head of                              0   1  \
0      id=82802525263321657348  ||   
1      id=82802525128797741057  ||   
2     id=828025025087815274496  ||   
3        id=828025006626058241  ||   
4        id=828024709761658880  ||   
...                        ...  ..   
5830     id=943971610670829570  ||   
5831     id=943971502784991232  ||   
5832   id=94393971433847447552  ||   
5833     id=943971418001281025  ||   
5834     id=943971377190760454  ||   

                                                      2   3    4  
0     Ismael is egocentric because she goes crazy if...  ||  0.0  
1     ...it took a while for someone to come out who...  ||  0.0  
2     (This is not a political and reasoned speech, ...  ||  0.0  
3     Very despised,siiii,but all dressed in Spanish...  ||  1.0  
4     faggot explain to me why sometimes I don't und...  ||  1.0  
...                                                 ...  ..  ...  
5830  Whether pro-independence or constituti

In [23]:
#drop the columns not needed
spanishtrainingSet.drop(columns=0, inplace=True)
spanishtrainingSet.drop(columns=1, inplace=True)
spanishtrainingSet.drop(columns=3,inplace=True)

In [24]:
#rename the column 2 to 'Tweet' and column 4 to 'Label
spanishtrainingSet=spanishtrainingSet.rename(columns={2: 'Tweet'})
spanishtrainingSet=spanishtrainingSet.rename(columns={4: 'Label'})

In [25]:
spanishtrainingSet

,Tweet,Label
0,Ismael is egocentric because she goes crazy if...,0.0
1,...it took a while for someone to come out who...,0.0
2,"(This is not a political and reasoned speech, ...",0.0
3,"Very despised,siiii,but all dressed in Spanish...",1.0
4,faggot explain to me why sometimes I don't und...,1.0
...,...,...
5830,"Whether pro-independence or constitutionalist,...",0.0
5831,@Bernithedude Key of government of what if the...,0.0
5832,@13Pardis @guenhwyvarblack Pugs are an ultra f...,0.0
5833,Applause in Colau's party when she announces i...,0.0


In [26]:
spanishtrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5835 entries, 0 to 5834
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Tweet   5835 non-null   object 
 1   Label   5835 non-null   float64
dtypes: float64(1), object(1)
memory usage: 91.3+ KB


**Remove all links**

In [27]:
#function remove_urls cleans the Urls from the Tweets and put in a seperate column
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

spanishtrainingSet["tweet_without_url"] = spanishtrainingSet["Tweet"].apply(lambda text: remove_urls(text))

In [28]:
spanishtrainingSet.head()

,Tweet,Label,tweet_without_url
0,Ismael is egocentric because she goes crazy if...,0.0,Ismael is egocentric because she goes crazy if...
1,...it took a while for someone to come out who...,0.0,...it took a while for someone to come out who...
2,"(This is not a political and reasoned speech, ...",0.0,"(This is not a political and reasoned speech, ..."
3,"Very despised,siiii,but all dressed in Spanish...",1.0,"Very despised,siiii,but all dressed in Spanish..."
4,faggot explain to me why sometimes I don't und...,1.0,faggot explain to me why sometimes I don't und...


**Load Italian Dataset and Clean it **

In [29]:
#load and read the training TEXT From file
italiantrainingSet = pd.read_csv('drive/MyDrive/italianDeeplTrans.txt', on_bad_lines='skip', sep='\t')

In [30]:
italiantrainingSet.head()

,id text hs stereotype
0,"2066 This is also terrorism, to put people in ..."
1,2045 @user @user in fact as long as they made ...
2,"6111 Corriere: Tangents, Mafia Capitale forgot..."
3,"1259 @user one by one, because when the Israel..."
4,949 Fun of the day? Finding Italian patriots ...


In [31]:
italiantrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5892 entries, 0 to 5891
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id text hs stereotype  5892 non-null   object
dtypes: object(1)
memory usage: 46.2+ KB


#Delete the last entry aka stereotype and then check whether the new last is hate speech (1) or not (0). Make a new column and document

In [32]:
#drop the columns not needed
#italiantrainingSet.drop(columns='id', inplace=True)
#italiantrainingSet.drop(columns='stereotype', inplace=True)

In [33]:
#rename the column text to 'Tweet' and column hs to 'Label
#italiantrainingSet=italiantrainingSet.rename(columns={'text ': 'Tweet'})
#italiantrainingSet=italiantrainingSet.rename(columns={'hs': 'Label'})
italiantrainingSet=italiantrainingSet.rename(columns={'id text hs stereotype': 'Tweet'})

In [34]:
italiantrainingSet['Tweet'] = italiantrainingSet['Tweet'].apply(lambda x: ' '.join(x.split()[:-1]))

In [35]:
def check_for_hate(row):
    if row.endswith("1"):
        return 1
    else:
        return 0

In [36]:
italiantrainingSet['Label'] = italiantrainingSet['Tweet'].apply(check_for_hate)

In [37]:
italiantrainingSet['Tweet'] = italiantrainingSet['Tweet'].apply(lambda x: ' '.join(x.split()[:-1]))

In [38]:
italiantrainingSet.head()

,Tweet,Label
0,"2066 This is also terrorism, to put people in ...",0
1,2045 @user @user in fact as long as they made ...,0
2,"6111 Corriere: Tangents, Mafia Capitale forgot...",0
3,"1259 @user one by one, because when the Israel...",0
4,949 Fun of the day? Finding Italian patriots r...,0


In [39]:
italiantrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5892 entries, 0 to 5891
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   5892 non-null   object
 1   Label   5892 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 92.2+ KB


In [40]:
count = (italiantrainingSet['Label'] == 1).sum()

# The condition (df['Label'] == 1) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 1 in 'Label' column:", count)

Number of rows with value 1 in 'Label' column: 1931


In [41]:
countZero = (italiantrainingSet['Label'] == 0).sum()

# The condition (df['Label'] == 0) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 0 in 'Label' column:", countZero)

Number of rows with value 0 in 'Label' column: 3961


In [42]:
#function remove_urls cleans the Urls from the Tweets and put in a seperate column
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

italiantrainingSet["tweet_without_url"] = italiantrainingSet["Tweet"].apply(lambda text: remove_urls(text))

In [43]:
italiantrainingSet.head()

,Tweet,Label,tweet_without_url
0,"2066 This is also terrorism, to put people in ...",0,"2066 This is also terrorism, to put people in ..."
1,2045 @user @user in fact as long as they made ...,0,2045 @user @user in fact as long as they made ...
2,"6111 Corriere: Tangents, Mafia Capitale forgot...",0,"6111 Corriere: Tangents, Mafia Capitale forgot..."
3,"1259 @user one by one, because when the Israel...",0,"1259 @user one by one, because when the Israel..."
4,949 Fun of the day? Finding Italian patriots r...,0,949 Fun of the day? Finding Italian patriots r...


In [44]:
italiantrainingSet["tweet_without_url"] = italiantrainingSet["tweet_without_url"].str[4:]

In [45]:
italiantrainingSet

,Tweet,Label,tweet_without_url
0,"2066 This is also terrorism, to put people in ...",0,"This is also terrorism, to put people in a st..."
1,2045 @user @user in fact as long as they made ...,0,@user @user in fact as long as they made mone...
2,"6111 Corriere: Tangents, Mafia Capitale forgot...",0,"Corriere: Tangents, Mafia Capitale forgottenM..."
3,"1259 @user one by one, because when the Israel...",0,"@user one by one, because when the Israelite ..."
4,949 Fun of the day? Finding Italian patriots r...,0,Fun of the day? Finding Italian patriots ranti...
...,...,...,...
5887,9340 Nation states must be ready to give up th...,0,Nation states must be ready to give up their ...
5888,9121 Germany's interior minister #HorstSeehofe...,0,"Germany's interior minister #HorstSeehofer,is..."
5889,8549 #Salvini: Too many people in Italy have g...,0,#Salvini: Too many people in Italy have gone ...
5890,9240 @user @user Those who jubilate in good fa...,0,@user @user Those who jubilate in good faith ...


**Merge the Data**

In [46]:
germantrainingSet.shape

(4816, 3)

In [47]:
#drop the columns not needed
germantrainingSet.drop(columns='Tweet', inplace=True)
#germantrainingSet.drop(columns='tweet_w/o_url', inplace=True)
#germantrainingSet.drop(columns='tweet_lem', inplace=True)

In [48]:
germantrainingSet.head()

,Label,tweet_without_url
0,0,"@corinnamilborn Dear Corinna, we'd love to hav..."
1,0,@Martin28a You are indeed correct. Our tweet w...
2,0,@ahrens_theo happy greetings from the most bea...
3,0,@dushanwegner Yanks would have voted for anyth...
4,0,@spdde not a reliable negotiating partner. Rec...


In [49]:
#figure out the shape of spanish set
spanishtrainingSet.shape

(5835, 3)

In [50]:
#drop the columns not needed
spanishtrainingSet.drop(columns='Tweet', inplace=True)
#spanishtrainingSet.drop(columns='tweet_w/o_url', inplace=True)
#spanishtrainingSet.drop(columns='tweet_lem', inplace=True)

In [51]:
spanishtrainingSet.head()

,Label,tweet_without_url
0,0.0,Ismael is egocentric because she goes crazy if...
1,0.0,...it took a while for someone to come out who...
2,0.0,"(This is not a political and reasoned speech, ..."
3,1.0,"Very despised,siiii,but all dressed in Spanish..."
4,1.0,faggot explain to me why sometimes I don't und...


In [52]:
#figure out the shape of italian set
italiantrainingSet.shape

(5892, 3)

In [53]:
#drop the columns not needed
italiantrainingSet.drop(columns='Tweet', inplace=True)
#italiantrainingSet.drop(columns='tweet_w/o_url', inplace=True)
#italiantrainingSet.drop(columns='tweet_lem', inplace=True)

In [54]:
#rename the column 2 to 'Tweet' and column 4 to 'Label
italiantrainingSet=italiantrainingSet.rename(columns={1: 'tweet_without_url'})


In [55]:
italiantrainingSet.head()

,Label,tweet_without_url
0,0,"This is also terrorism, to put people in a st..."
1,0,@user @user in fact as long as they made mone...
2,0,"Corriere: Tangents, Mafia Capitale forgottenM..."
3,0,"@user one by one, because when the Israelite ..."
4,0,Fun of the day? Finding Italian patriots ranti...


In [56]:
germanspanish_merged = pd.merge(germantrainingSet, spanishtrainingSet, how="outer", on=["Label", "tweet_without_url"])

In [57]:
germanspanish_merged.shape

(10651, 2)

In [58]:
germanspanish_merged.head()

,Label,tweet_without_url
0,0.0,"@corinnamilborn Dear Corinna, we'd love to hav..."
1,0.0,@Martin28a You are indeed correct. Our tweet w...
2,0.0,@ahrens_theo happy greetings from the most bea...
3,0.0,@dushanwegner Yanks would have voted for anyth...
4,0.0,@spdde not a reliable negotiating partner. Rec...


In [59]:
print(germanspanish_merged)

       Label                                  tweet_without_url
0        0.0  @corinnamilborn Dear Corinna, we'd love to hav...
1        0.0  @Martin28a You are indeed correct. Our tweet w...
2        0.0  @ahrens_theo happy greetings from the most bea...
3        0.0  @dushanwegner Yanks would have voted for anyth...
4        0.0  @spdde not a reliable negotiating partner. Rec...
...      ...                                                ...
10646    0.0  Whether pro-independence or constitutionalist,...
10647    0.0  @Bernithedude Key of government of what if the...
10648    0.0  @13Pardis @guenhwyvarblack Pugs are an ultra f...
10649    0.0  Applause in Colau's party when she announces i...
10650    0.0  @Rafa_Hernando The Constitution is pro-indepen...

[10651 rows x 2 columns]


In [60]:
germanspanishitalian_merged = pd.merge(germanspanish_merged, italiantrainingSet, how="outer", on=["Label", "tweet_without_url"])

In [61]:
germanspanishitalian_merged.shape

(16543, 2)

In [62]:
print(germanspanishitalian_merged)

       Label                                  tweet_without_url
0        0.0  @corinnamilborn Dear Corinna, we'd love to hav...
1        0.0  @Martin28a You are indeed correct. Our tweet w...
2        0.0  @ahrens_theo happy greetings from the most bea...
3        0.0  @dushanwegner Yanks would have voted for anyth...
4        0.0  @spdde not a reliable negotiating partner. Rec...
...      ...                                                ...
16538    0.0   Nation states must be ready to give up their ...
16539    0.0   Germany's interior minister #HorstSeehofer,is...
16540    0.0   #Salvini: Too many people in Italy have gone ...
16541    0.0   @user @user Those who jubilate in good faith ...
16542    0.0   Young Christians in #Ethiopia are induced by ...

[16543 rows x 2 columns]


In [63]:
germanspanishitalian_merged.head()

,Label,tweet_without_url
0,0.0,"@corinnamilborn Dear Corinna, we'd love to hav..."
1,0.0,@Martin28a You are indeed correct. Our tweet w...
2,0.0,@ahrens_theo happy greetings from the most bea...
3,0.0,@dushanwegner Yanks would have voted for anyth...
4,0.0,@spdde not a reliable negotiating partner. Rec...


Rename Label to 'label'

In [64]:
germanspanishitalian_merged = germanspanishitalian_merged.rename(columns={'Label': 'label'})

In [65]:
germanspanishitalian_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16543 entries, 0 to 16542
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   label              16543 non-null  float64
 1   tweet_without_url  16543 non-null  object 
dtypes: float64(1), object(1)
memory usage: 387.7+ KB


In [66]:
count = (germanspanishitalian_merged['label'] == 1).sum()

# The condition (df['Label'] == 1) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 1 in 'Label' column:", count)

Number of rows with value 1 in 'Label' column: 4457


In [67]:
countZero = (germanspanishitalian_merged['label'] == 0).sum()

# The condition (df['Label'] == 0) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 0 in 'Label' column:", countZero)

Number of rows with value 0 in 'Label' column: 12086


In [68]:
#germanspanishitalian_merged

In [69]:
X = germanspanishitalian_merged["tweet_without_url"].values.tolist()

In [70]:
y = germanspanishitalian_merged['label'].values.tolist()

**Split the text**

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 0)

In [72]:
train_texts = X_train
train_labels = y_train

test_texts = X_test
test_labels = y_test

In [73]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(13234, 13234, 3309, 3309)

In [74]:
train_labels[0], train_texts[0]

(0.0,
 '@dvallespi You fascists have to learn to respect that Catalonia has two languages of its own and to stop this sick addiction to indoctrinate children.')

In [75]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [76]:
#import svm model and metrics
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [77]:
#model = LogisticRegression(max_iter=1000).fit(X_train, train_labels)
#SVM BASE LINE
#model = SVC(kernel='linear', C=100, gamma='auto').fit(X_train, train_labels)
model = SVC(kernel='linear', gamma='auto').fit(X_train, train_labels)
predictions = model.predict(X_test)

In [78]:
print(classification_report(test_labels, predictions))

              precision    recall  f1-score   support

         0.0       0.82      0.92      0.87      2412
         1.0       0.68      0.45      0.55       897

    accuracy                           0.80      3309
   macro avg       0.75      0.69      0.71      3309
weighted avg       0.78      0.80      0.78      3309



In [79]:
#calculate F1 score
from sklearn.metrics import f1_score
f1_score(test_labels, predictions)

0.5465505693235096

In [80]:
from sklearn.metrics import matthews_corrcoef
#calculate Matthews correlation coefficient
matthews_corrcoef(test_labels, predictions)

0.4359832082173881

In [81]:
# This is the name of the BERT model that we want to use.
model_name = 'distilbert-base-cased'

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'

# This is the maximum number of tokens in any document sent to BERT.
max_length = 512

# This is the name of the directory where we'll save our model. You can name it whatever you want.
cached_model_directory_name = 'Masterdistilbert-deepl-model-saved'

In [82]:
from transformers import BertForSequenceClassification, BertTokenizer
#tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

In [83]:
train_labels


[0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0

In [84]:
unique_labels = set(label for label in train_labels)
label2id = {label: id for id, label in enumerate(unique_labels)}
id2label = {id: label for label, id in label2id.items()}

In [85]:
label2id.keys()

dict_keys([0.0, 1.0])

In [86]:
id2label.keys()

dict_keys([0, 1])

In [87]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
test_encodings  = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)

train_labels_encoded = [label2id[y] for y in train_labels]
test_labels_encoded  = [label2id[y] for y in test_labels]

In [88]:
set(train_labels_encoded)

{0, 1}

In [89]:
set(test_labels_encoded)

{0, 1}

In [90]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [91]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [92]:
# The model_name needs to match the name used for the tokenizer above.
#model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bia

In [93]:
#!pip install --upgrade accelerate

In [94]:
training_args = TrainingArguments(
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)

In [95]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [96]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function
)

In [97]:
torch.cuda.empty_cache()

In [98]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.612100,0.554068,0.728921
200,0.518400,0.593532,0.735872
300,0.515400,0.476577,0.772439
400,0.511400,0.462389,0.785131
500,0.482200,0.454841,0.781807
600,0.489600,0.441985,0.783318
700,0.521600,0.443500,0.788154
800,0.466600,0.439581,0.786038
900,0.433300,0.474637,0.791176
1000,0.468200,0.437885,0.796011


TrainOutput(global_step=3310, training_loss=0.40527151689788726, metrics={'train_runtime': 440.416, 'train_samples_per_second': 60.098, 'train_steps_per_second': 7.516, 'total_flos': 1581874964573904.0, 'train_loss': 0.40527151689788726, 'epoch': 2.0})

In [99]:
trainer.save_model(cached_model_directory_name)

In [100]:
trainer.evaluate()

{'eval_loss': 0.48843029141426086,
 'eval_accuracy': 0.8114233907524931,
 'eval_runtime': 5.7801,
 'eval_samples_per_second': 572.486,
 'eval_steps_per_second': 28.719,
 'epoch': 2.0}

In [101]:
predicted_results = trainer.predict(test_dataset)

In [102]:
predicted_results.predictions.shape

(3309, 2)

In [103]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

In [104]:
len(predicted_labels)

3309

Print the results for Distilbert + DeeplTrans dataset

In [105]:
print(classification_report(test_labels,
                            predicted_labels))

              precision    recall  f1-score   support

         0.0       0.85      0.90      0.87      2412
         1.0       0.68      0.58      0.62       897

    accuracy                           0.81      3309
   macro avg       0.77      0.74      0.75      3309
weighted avg       0.80      0.81      0.81      3309



In [106]:
#calculate F1 score
from sklearn.metrics import f1_score
f1_score(test_labels, predicted_labels)

0.6240963855421686

In [107]:
from sklearn.metrics import matthews_corrcoef
#calculate Matthews correlation coefficient
matthews_corrcoef(test_labels, predicted_labels)

0.5022427816998448